In [2]:
# %load retinanet_train.py
#!/usr/bin/env python

# In[1]:

print("Start importing package...")
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
import mymodel
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader
from mydataloader import MyDataset,RandomTransformer
import coco_eval
import csv_eval
#import visdom
#vis = visdom.Visdom(env='retinanet_seresnextTest')
from tensorboardX import SummaryWriter
assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#参数
NAME="RSNA"
DATA_PATH = "/data/krf/dataset"
CSV_TRAINS = [DATA_PATH + "/test_csv_train0.csv",DATA_PATH + "/test_csv_train1.csv",DATA_PATH + "/test_csv_train2.csv",DATA_PATH + "/test_csv_train3.csv"]
CSV_VALS = [DATA_PATH + "/test_csv_val0.csv",DATA_PATH + "/test_csv_val1.csv",DATA_PATH + "/test_csv_val2.csv",DATA_PATH + "/test_csv_val3.csv"]
CSV_CLASSES = DATA_PATH + "/test_classes.csv"
DEPTH = 50
EPOCHS = 20
BATCH_SIZE=2
VAL_SIZE = 30
#TRAIN_SIZE = 100
#数据预处理
import csv
import random
from tqdm import tqdm

Start importing package...


Using TensorFlow backend.


CUDA available: True


In [4]:
# with open(DATA_PATH+"/stage_1_train_labels.csv") as f:
#     reader = csv.reader(f)
#     rows=[row for row in  reader]
#     rows = rows[1:]
#     random.shuffle(rows)
#     for row in rows:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[1] == '' and row[2] == '' and row[3] == '' and row[4] == '':
#             row[5] = ''
#         else:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#     val_rows = rows[:VAL_SIZE]
#     train_rows = rows[VAL_SIZE:]
#     print(len(val_rows),len(train_rows))
#     with open(CSV_TRAIN,'w') as f2:
#         write = csv.writer(f2)
#         write.writerows(train_rows)
#         print("csv_train 写入完毕")
#     with open(CSV_VAL,'w') as f3:
#         write = csv.writer(f3)
#         write.writerows(val_rows)
#         print("csv_val 写入完毕")

# with open(CSV_CLASSES,'w') as f:
#     write = csv.writer(f)
    
#     row = ['0','0']
#     write.writerow(row)
#     row = ['1','1']
#     write.writerow(row)
#     row = ['2','2']
#     write.writerow(row)
#     print("csv_classes 写入完毕")



# # #每次跑这个函数之前需要先删除之前的
# CSV_TRAINS = [DATA_PATH + "/csv_train.csv"]
# CSV_VALS = [DATA_PATH + "/csv_val"]
# df = pd.read_csv(DATA_PATH+"/stage_1_train_labels.csv")
# ddf = pd.read_csv(DATA_PATH+'/stage_1_detailed_class_info.csv')
# train_images = os.listdir(DATA_PATH+"/stage_1_train_images")[:500]

# random.shuffle(train_images)#打乱图片顺序
# count = 0
# pos_cnt_train = [0,0,0,0]
# pos_cnt_val = [0,0,0,0]
# class_dict = {'Normal':0,'Lung Opacity':1,'No Lung Opacity / Not Normal':2}
# for img_name in tqdm(train_images):
#     results = df[df['patientId']==img_name.split('.')[0]].values
#     detail = ddf[df['patientId']==img_name.split('.')[0]].values
#     for row in results:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[5] == 1:
#             pos_cnt_val[0] += 1
#         if row[1] >= 0 and row[1] <= 1024:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#         else:
#             row[1] = ''
#             row[2] = ''
#             row[3] = ''
#             row[4] = ''
#             row[5] = class_dict[detail[0,1]]
#         with open(CSV_VALS[0],'a') as f:
#             write = csv.writer(f)
#             write.writerow(row)
#         for i in range(4):
#             if count % 4 == i:
#                 continue
#             else:
#                 with open(CSV_TRAINS[0],'a') as f:
#                     write = csv.writer(f)
#                     write.writerow(row) 
#     count += 1
# print(pos_cnt_train,pos_cnt_val) 

# #每次跑这个函数之前需要先删除之前的
CSV_TRAINS = [DATA_PATH + "/csv_train.csv"]
CSV_VALS = [DATA_PATH + "/csv_val.csv"]
df = pd.read_csv(DATA_PATH+"/stage_1_train_labels.csv")
ddf = pd.read_csv(DATA_PATH+'/stage_1_detailed_class_info.csv')
train_images = os.listdir(DATA_PATH+"/stage_1_train_images")

random.shuffle(train_images)#打乱图片顺序
count = 0
pos_cnt_train = [0,0,0,0]
pos_cnt_val = [0,0,0,0]
class_dict = {'Normal':0,'Lung Opacity':1,'No Lung Opacity / Not Normal':2}
for img_name in tqdm(train_images):
    results = df[df['patientId']==img_name.split('.')[0]].values
    detail = ddf[df['patientId']==img_name.split('.')[0]].values
    if count < 1400:
        file = CSV_VALS[0]
    else:
        file = CSV_TRAINS[0]
    for row in results:
        row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
        if row[5] == 1:
            pos_cnt_val[0] += 1
        if row[1] >= 0 and row[1] <= 1024:
            row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
            row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
        else:
            row[1] = ''
            row[2] = ''
            row[3] = ''
            row[4] = ''
            row[5] = class_dict[detail[0,1]]
        with open(file,'a') as f:
            write = csv.writer(f)
            write.writerow(row)
        
    count += 1
print(pos_cnt_train,pos_cnt_val)  

100%|██████████| 25684/25684 [02:17<00:00, 189.57it/s]

[0, 0, 0, 0] [8964, 0, 0, 0]


In [3]:

print("Loading data...")

#%time
#制作数据loader
dataset_train = []
dataset_val = []

for i in range(2): 
    dataset_train.append(MyDataset(train_file=CSV_TRAINS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(),RandomTransformer(), Resizer()])))
    dataset_val.append(MyDataset(train_file=CSV_VALS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Resizer()])))

#每次的sampler的参数：来源、batchsize、是否抛弃最后一层？？？

# num_workers 同时工作的组？collater:校验用的吧
dataloader_train = []
dataloader_val = []

for i in range(2):
    sampler = AspectRatioBasedSampler(dataset_train[i], batch_size=BATCH_SIZE, drop_last=False)
    sampler_val = AspectRatioBasedSampler(dataset_val[i], batch_size=1, drop_last=False)
    dataloader_train.append(DataLoader(dataset_train[i], num_workers=1, collate_fn=collater, batch_sampler=sampler))
    dataloader_val.append(DataLoader(dataset_val[i], num_workers=1, collate_fn=collater, batch_sampler=sampler_val))



Loading data...


In [4]:
retinanets = []
# Create the model
# if DEPTH == 18:
#     retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 34:
#     retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 50:
#     for i in range(4):
#         retinanets.append(model.resnet50(num_classes=dataset_train[i].num_classes(), pretrained=True))
# elif DEPTH == 101:
#     for i in range(4):
#         retinanets.append(model.resnet101(num_classes=dataset_train[i].num_classes(), pretrained=True)) 
# elif DEPTH == 152:
#     for i in range(4):
#         retinanets.append(model.resnet152(num_classes=dataset_train[i].num_classes(), pretrained=True))
# else:
#     raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')
#retinanet = torch.load('weights/RSNA_retinanet_5.pt')
for i in range(2):
    retinanets.append(mymodel.se_resnext50_32x4d(num_classes=dataset_train[i].num_classes()))
optimizer = []
scheduler = []
loss_hist = []
classloss_hist = []
regressloss_hist = []
wholeclassloss_hist = []

writer = SummaryWriter()
for i in range(2):
    retinanets[i] = retinanets[i].cuda()
    #变成并行
    retinanets[i] = torch.nn.DataParallel(retinanets[i]).cuda()
    #训练模式
    retinanets[i].training = True
    #学习率0.00001  
    optimizer.append(optim.Adam(retinanets[i].parameters(), lr=1e-4))
    #如果3个epoch损失没有减少则降低学习率
    scheduler.append(optim.lr_scheduler.ReduceLROnPlateau(optimizer[i], patience=2, verbose=True))
    # TODO 这是干什么 deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：这里是定义了一个500的队列
    loss_hist.append(collections.deque(maxlen=500))
    classloss_hist.append(collections.deque(maxlen=500))
    regressloss_hist.append(collections.deque(maxlen=500))
    wholeclassloss_hist.append(collections.deque(maxlen=500))
    print('Num training images: {}'.format(len(dataset_train[i])))
# In[5]:



Num training images: 300
Num training images: 280


In [5]:
VAL_SIZE = 15
count =[0,0,0,0]
import traceback
for epoch_num in range(EPOCHS):

    for i in range(2):
        retinanets[i].train()
        retinanets[i].module.freeze_bn()
        epoch_loss = []
        
        for iter_num, data in enumerate(dataloader_train[i]):
            if 1 == 1:
            #try:
                optimizer[i].zero_grad()

                focal_loss, whole_class_loss = retinanets[i]([Variable(data['img'].cuda().float()), Variable(data['annot'].cuda())])
                classification_loss, regression_loss = focal_loss
                classification_loss = classification_loss.mean()
                regression_loss = regression_loss.mean()
                whole_class_loss = whole_class_loss.mean()
                loss = classification_loss + regression_loss + whole_class_loss

                if bool(loss == 0):
                    continue
                #反向传播？
                loss.backward()

                #这是干嘛？？TODO
                torch.nn.utils.clip_grad_norm_(retinanets[i].parameters(), 0.1)

                #这?TODO
                optimizer[i].step()

                loss_hist[i].append(float(loss))
                classloss_hist[i].append(float(classification_loss))
                regressloss_hist[i].append(float(regression_loss))
                wholeclassloss_hist[i].append(float(whole_class_loss))
                epoch_loss.append(float(loss))
                if count[i] % 10 == 0:
                    print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Whole Class loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, np.mean(classloss_hist[i]), np.mean(regressloss_hist[i]), np.mean(wholeclassloss_hist[i]), np.mean(loss_hist[i])))
    #                 vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(loss_hist[i])]), win='train loss '+str(i), update='append' ,opts={'title':'train loss '+str(i)})
    #                 vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(classloss_hist[i])]), win='classification loss '+str(i), update='append' ,opts={'title':'classification loss '+str(i)})
    #                 vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(regressloss_hist[i])]), win='regression loss '+str(i), update='append' ,opts={'title':'regression loss '+str(i)})
    #                 vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(wholeclassloss_hist[i])]), win='whole class loss '+str(i), update='append' ,opts={'title':'whole class loss '+str(i)})
                    writer.add_scalar('Model_{}/Loss'.format(i), np.mean(loss_hist[i]), count[i])
                    writer.add_scalar('Model_{}/Classification Loss'.format(i), np.mean(classloss_hist[i]), count[i])
                    writer.add_scalar('Model_{}/Regression Loss'.format(i), np.mean(regressloss_hist[i]), count[i])
                    writer.add_scalar('Model_{}/Whole Class Loss'.format(i), np.mean(wholeclassloss_hist[i]), count[i])
                count[i] += 1
                #vis.save(['retinanet_seresnextTest'])

                del classification_loss
                del regression_loss
#                 print(count[i])
#                 if count[i] % VAL_SIZE == 0:
#                     print("Evaluating dataset")
#                     retinanets[i].eval()
#                     mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
#                     vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([mAP[0][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
#                     vis.save(['retinanet_seresnextTest'])
#                     #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
#                     retinanets[i].train()
#             except Exception as e:
#                 print(e)
#                 continue


        print("Evaluating dataset")
        retinanets[i].eval()
    #     for index,data in enumerate(dataloader_val):
    #         #data = dataset[index]
    #         scale = data['scale']
    #         # run network
    #         #scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    #         print(Variable(data['img'].cuda()))
    #         scores, labels, boxes = retinanet(Variable(data['img'].cuda()))
    #         #mAP = csv_eval.evaluate(dataset_val,retinanet)
#         try:
        if 1 == 1:
            mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
            writer.add_scalar('Model_{}/mAP'.format(i), mAP[1][0], epoch_num)
#             vis.line(X=torch.Tensor([epoch_num]), Y=torch.Tensor([mAP[1][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
#             vis.save(['retinanet_seresnextTest'])
#         except Exception as e:
#             print(e)
#             print( 'traceback.print_exc():', traceback.print_exc())
#             continue
        #这一步也看不懂？？TODO 
        scheduler[i].step(np.mean(epoch_loss))

        #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
    #     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

for i in range(2):
    retinanets[i].eval()
    #torch.save(retinanets[i], 'weights_stage1/{}_model_final_seresnext101.pt'.format(i))
writer.export_scalars_to_json("./test.json")
writer.close()



/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/parallel/_functions.py:58: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch: 0 | Iteration: 0 | Classification loss: 0.12567 | Regression loss: 0.00000 | Whole Class loss: 1.10748 | Running loss: 1.23316
Epoch: 0 | Iteration: 10 | Classification loss: 0.32478 | Regression loss: 0.24187 | Whole Class loss: 1.26330 | Running loss: 1.82996
Epoch: 0 | Iteration: 20 | Classification loss: 0.31313 | Regression loss: 0.25089 | Whole Class loss: 1.18595 | Running loss: 1.74997
Epoch: 0 | Iteration: 30 | Classification loss: 0.23304 | Regression loss: 0.18681 | Whole Class loss: 1.11527 | Running loss: 1.53512
Epoch: 0 | Iteration: 40 | Classification loss: 0.19718 | Regression loss: 0.15429 | Whole Class loss: 1.12135 | Running loss: 1.47282
Epoch: 0 | Iteration: 50 | Classification loss: 0.21105 | Regression loss: 0.16440 | Whole Class loss: 1.16216 | Running loss: 1.53762
Epoch: 0 | Iteration: 60 | Classification loss: 0.21702 | Regression loss: 0.17112 | Whole Class loss: 1.13727 | Running loss: 1.52541
Epoch: 0 | Iteration: 70 | Classification loss: 0.21142 

Epoch: 2 | Iteration: 20 | Classification loss: 0.22890 | Regression loss: 0.19484 | Whole Class loss: 1.12536 | Running loss: 1.54911
Epoch: 2 | Iteration: 30 | Classification loss: 0.22970 | Regression loss: 0.19542 | Whole Class loss: 1.12156 | Running loss: 1.54669
Epoch: 2 | Iteration: 40 | Classification loss: 0.22852 | Regression loss: 0.19379 | Whole Class loss: 1.12035 | Running loss: 1.54266
Epoch: 2 | Iteration: 50 | Classification loss: 0.22367 | Regression loss: 0.18981 | Whole Class loss: 1.10990 | Running loss: 1.52338
Epoch: 2 | Iteration: 60 | Classification loss: 0.22237 | Regression loss: 0.18842 | Whole Class loss: 1.11854 | Running loss: 1.52933
Epoch: 2 | Iteration: 70 | Classification loss: 0.22377 | Regression loss: 0.18961 | Whole Class loss: 1.12418 | Running loss: 1.53756
Epoch: 2 | Iteration: 80 | Classification loss: 0.21946 | Regression loss: 0.18601 | Whole Class loss: 1.11825 | Running loss: 1.52371
Epoch: 2 | Iteration: 90 | Classification loss: 0.21668

Epoch: 4 | Iteration: 40 | Classification loss: 0.21644 | Regression loss: 0.18678 | Whole Class loss: 1.09678 | Running loss: 1.50000
Epoch: 4 | Iteration: 50 | Classification loss: 0.21259 | Regression loss: 0.18343 | Whole Class loss: 1.09191 | Running loss: 1.48792
Epoch: 4 | Iteration: 60 | Classification loss: 0.21566 | Regression loss: 0.18625 | Whole Class loss: 1.09355 | Running loss: 1.49546
Epoch: 4 | Iteration: 70 | Classification loss: 0.21736 | Regression loss: 0.18880 | Whole Class loss: 1.09230 | Running loss: 1.49847
Epoch: 4 | Iteration: 80 | Classification loss: 0.21551 | Regression loss: 0.18686 | Whole Class loss: 1.09029 | Running loss: 1.49267
Epoch: 4 | Iteration: 90 | Classification loss: 0.21566 | Regression loss: 0.18696 | Whole Class loss: 1.09483 | Running loss: 1.49745
Epoch: 4 | Iteration: 100 | Classification loss: 0.21605 | Regression loss: 0.18663 | Whole Class loss: 1.10210 | Running loss: 1.50478
Epoch: 4 | Iteration: 110 | Classification loss: 0.213

Epoch: 6 | Iteration: 50 | Classification loss: 0.20782 | Regression loss: 0.18110 | Whole Class loss: 1.09520 | Running loss: 1.48412
Epoch: 6 | Iteration: 60 | Classification loss: 0.20629 | Regression loss: 0.18005 | Whole Class loss: 1.09691 | Running loss: 1.48326
Epoch: 6 | Iteration: 70 | Classification loss: 0.20314 | Regression loss: 0.17773 | Whole Class loss: 1.09030 | Running loss: 1.47117
Epoch: 6 | Iteration: 80 | Classification loss: 0.19921 | Regression loss: 0.17539 | Whole Class loss: 1.08611 | Running loss: 1.46071
Epoch: 6 | Iteration: 90 | Classification loss: 0.19787 | Regression loss: 0.17351 | Whole Class loss: 1.08949 | Running loss: 1.46088
Epoch: 6 | Iteration: 100 | Classification loss: 0.19626 | Regression loss: 0.17156 | Whole Class loss: 1.08900 | Running loss: 1.45681
Epoch: 6 | Iteration: 110 | Classification loss: 0.20166 | Regression loss: 0.17634 | Whole Class loss: 1.10111 | Running loss: 1.47911
Epoch: 6 | Iteration: 120 | Classification loss: 0.20

...

Epoch: 10 | Iteration: 70 | Classification loss: 0.19317 | Regression loss: 0.16375 | Whole Class loss: 1.02156 | Running loss: 1.37848
Epoch: 10 | Iteration: 80 | Classification loss: 0.19320 | Regression loss: 0.16344 | Whole Class loss: 1.01688 | Running loss: 1.37352
Epoch: 10 | Iteration: 90 | Classification loss: 0.19177 | Regression loss: 0.16228 | Whole Class loss: 1.02025 | Running loss: 1.37430
Epoch: 10 | Iteration: 100 | Classification loss: 0.18884 | Regression loss: 0.16066 | Whole Class loss: 1.01750 | Running loss: 1.36700
Epoch: 10 | Iteration: 110 | Classification loss: 0.18787 | Regression loss: 0.16037 | Whole Class loss: 1.01595 | Running loss: 1.36419
Epoch: 10 | Iteration: 120 | Classification loss: 0.19044 | Regression loss: 0.16300 | Whole Class loss: 1.01397 | Running loss: 1.36741
Epoch: 10 | Iteration: 130 | Classification loss: 0.18919 | Regression loss: 0.16182 | Whole Class loss: 1.01411 | Running loss: 1.36511
Epoch: 10 | Iteration: 140 | Classification 

Epoch: 12 | Iteration: 80 | Classification loss: 0.18189 | Regression loss: 0.15637 | Whole Class loss: 0.98122 | Running loss: 1.31948
Epoch: 12 | Iteration: 90 | Classification loss: 0.17786 | Regression loss: 0.15305 | Whole Class loss: 0.97444 | Running loss: 1.30534
Epoch: 12 | Iteration: 100 | Classification loss: 0.17797 | Regression loss: 0.15302 | Whole Class loss: 0.97730 | Running loss: 1.30830
Epoch: 12 | Iteration: 110 | Classification loss: 0.17867 | Regression loss: 0.15345 | Whole Class loss: 0.97676 | Running loss: 1.30887
Epoch: 12 | Iteration: 120 | Classification loss: 0.17925 | Regression loss: 0.15377 | Whole Class loss: 0.96656 | Running loss: 1.29958
Epoch: 12 | Iteration: 130 | Classification loss: 0.18058 | Regression loss: 0.15504 | Whole Class loss: 0.97725 | Running loss: 1.31287
Epoch: 12 | Iteration: 140 | Classification loss: 0.17887 | Regression loss: 0.15353 | Whole Class loss: 0.97752 | Running loss: 1.30993
Evaluating dataset
300/300
mAP:
0: 0.0
1: 0

Epoch: 14 | Iteration: 90 | Classification loss: 0.17481 | Regression loss: 0.14971 | Whole Class loss: 0.96957 | Running loss: 1.29408
Epoch: 14 | Iteration: 100 | Classification loss: 0.17383 | Regression loss: 0.14859 | Whole Class loss: 0.96409 | Running loss: 1.28651
Epoch: 14 | Iteration: 110 | Classification loss: 0.17761 | Regression loss: 0.15195 | Whole Class loss: 0.97569 | Running loss: 1.30525
Epoch: 14 | Iteration: 120 | Classification loss: 0.17919 | Regression loss: 0.15390 | Whole Class loss: 0.97930 | Running loss: 1.31239
Epoch: 14 | Iteration: 130 | Classification loss: 0.17602 | Regression loss: 0.15152 | Whole Class loss: 0.97240 | Running loss: 1.29994
Epoch: 14 | Iteration: 140 | Classification loss: 0.17858 | Regression loss: 0.15362 | Whole Class loss: 0.97957 | Running loss: 1.31177
Evaluating dataset
300/300
mAP:
0: 0.0
1: 0.06930906063926254
2: 0.0
Epoch: 14 | Iteration: 0 | Classification loss: 0.19460 | Regression loss: 0.17098 | Whole Class loss: 0.94867

Epoch: 16 | Iteration: 90 | Classification loss: 0.17647 | Regression loss: 0.15142 | Whole Class loss: 0.96190 | Running loss: 1.28979
Epoch: 16 | Iteration: 100 | Classification loss: 0.17648 | Regression loss: 0.15156 | Whole Class loss: 0.96741 | Running loss: 1.29546
Epoch: 16 | Iteration: 110 | Classification loss: 0.17392 | Regression loss: 0.14911 | Whole Class loss: 0.96731 | Running loss: 1.29034
Epoch: 16 | Iteration: 120 | Classification loss: 0.17333 | Regression loss: 0.14834 | Whole Class loss: 0.96813 | Running loss: 1.28980
Epoch: 16 | Iteration: 130 | Classification loss: 0.17424 | Regression loss: 0.14883 | Whole Class loss: 0.96609 | Running loss: 1.28916
Epoch: 16 | Iteration: 140 | Classification loss: 0.17345 | Regression loss: 0.14880 | Whole Class loss: 0.96095 | Running loss: 1.28320
Evaluating dataset
300/300
mAP:
0: 0.0
1: 0.0873145974400663
2: 0.0
Epoch: 16 | Iteration: 0 | Classification loss: 0.19126 | Regression loss: 0.16902 | Whole Class loss: 0.94696 

Epoch: 18 | Iteration: 100 | Classification loss: 0.17204 | Regression loss: 0.14790 | Whole Class loss: 0.95637 | Running loss: 1.27632
Epoch: 18 | Iteration: 110 | Classification loss: 0.17253 | Regression loss: 0.14790 | Whole Class loss: 0.95884 | Running loss: 1.27928
Epoch: 18 | Iteration: 120 | Classification loss: 0.17165 | Regression loss: 0.14694 | Whole Class loss: 0.95788 | Running loss: 1.27646
Epoch: 18 | Iteration: 130 | Classification loss: 0.17395 | Regression loss: 0.14903 | Whole Class loss: 0.95510 | Running loss: 1.27808
Epoch: 18 | Iteration: 140 | Classification loss: 0.17456 | Regression loss: 0.14975 | Whole Class loss: 0.95514 | Running loss: 1.27945
Evaluating dataset
300/300
mAP:
0: 0.0
1: 0.08071697021766827
2: 0.0
Epoch: 18 | Iteration: 0 | Classification loss: 0.18095 | Regression loss: 0.16048 | Whole Class loss: 0.91961 | Running loss: 1.26104
Epoch: 18 | Iteration: 10 | Classification loss: 0.18061 | Regression loss: 0.16009 | Whole Class loss: 0.91617